In [189]:
# imports
# from openai import OpenAI
import statistics
import pandas as pd
import numpy as np
import json
import jsonlines

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from ctypes import alignment

In [190]:
# Open and read the JSON file
with open('C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\questions.json', 'r') as file:
    questions = json.load(file)

# this is a weird way to do this but
baseline = pd.json_normalize(questions, record_path=['baseline'])
graham = pd.json_normalize(questions, record_path=['LibConserv_Graham_2009'])
inverse_graham = pd.json_normalize(questions, record_path=['Inverse_LibConserv_Graham_2009'])

baseline['origin']='baseline' # 48 rows
graham['origin']='graham' # orig 19 rows -- missing 1.4 (gov), added to even to 20
inverse_graham['origin']='inverse' # 20 rows

prompts = pd.concat([baseline, graham, inverse_graham], ignore_index=True) # 88 rows!
prompts.rename(columns={'text': 'prompt', 'number': "prompt_number"}, inplace=True)
prompts = prompts.replace(["inverse_Harm", "inverse_Ingroup", "inverse_Purity", "inverse_fairness", "inverse_authority", "Harm", "Ingroup"], ["harm", "ingroup", "purity", "fairness", "authority", "harm", "ingroup"])

graham_prompts = prompts.loc[prompts['origin'] != 'baseline']
graham_prompts = graham_prompts.reset_index(drop=True)

In [191]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [192]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [193]:
prompts = graham_prompts

In [194]:
def jsonlines_to_df_results(filename):

  list_of_responses = []
  #Iterate over the json files
  with jsonlines.open(filename) as reader:
      #Iterate over the each line on the reader via enumerate
      for idx, obj in enumerate(reader):
        # print(obj['response']['body']['choices'][0]['message']['content'])
        list_of_responses.append(obj['response']['body']['choices'][0]['message']['content'])
  print(len(list_of_responses))
  print()

  df = pd.DataFrame(columns=["prompt", "alignment", "prompt_number", "category", "origin"])
  alignments = ["liberal", "conservative"]

  while len(list_of_responses) > df.shape[0]:
    for i, alignment in enumerate(alignments):
        if alignment == 'liberal':
          initial_prompt = initial_lib
        else:
          initial_prompt = initial_con

        for j, prompt in enumerate(prompts.prompt.to_list()):
            # print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")
            df.loc[len(df)] = [prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin']]

  df.insert(5, 'model', "gpt-4o-mini")
  df.insert(0, 'response', list_of_responses)

  return df

In [195]:
alpha = jsonlines_to_df_results(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_base\results_gpt4omini_temp1_graham_base_x15.jsonl')
alpha['temperature'] = 1.0

1200



In [196]:
beta = jsonlines_to_df_results(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_base\results_gpt4omini_temp2_graham_base_x15.jsonl')
beta['temperature'] = 2.0

1200



In [197]:
alpha.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,1.0
1,1,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,1.0
2,1,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,1.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,1.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,1.0


In [198]:
beta.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0


In [199]:
# alpha.to_csv(f'gpt4omini_temp1_base_results_all_RAW.csv', index = True)
# beta.to_csv(f'gpt4omini_temp2_base_results_all_RAW.csv', index = True)

### more process

In [200]:
# results_raw = pd.read_csv(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\code\gpt4omini_temp1_results_all_RAW.csv")

In [201]:
# display(results_raw)

In [202]:
results = beta

In [203]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0


In [204]:
results.response.unique

<bound method Series.unique of 0                                                       1
1       2\ResponseProv_des basesเตย conflict_xcompiler...
2                                                       2
3                                                       2
4                                                       5
                              ...                        
1195                                                    1
1196    5 берем นักลงทุนสัมพันธ์jsonัพ>(_sb}],imonials...
1197                                                    4
1198                                                    4
1199                                                    4
Name: response, Length: 1200, dtype: object>

In [205]:
responses = results['response'].values.tolist()

In [206]:
num = []
for x in responses:
    num.append(x[0])

In [207]:
results['response_processed'] = num

In [208]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature,response_processed
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,1
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,2
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,2
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5


In [209]:
results.response_processed.unique()

array(['1', '2', '5', '0', '4', '3', '('], dtype=object)

In [211]:
results['response_processed'] = pd.to_numeric(results.response_processed, errors='coerce').fillna(np.nan)

In [212]:
results.dtypes

response               object
prompt                 object
alignment              object
prompt_number         float64
category               object
origin                 object
model                  object
temperature           float64
response_processed    float64
dtype: object

In [213]:
# results["response_processed"] = results["response_processed"].astype("int64")
results["origin"] = results["origin"].astype("category")
results["category"] = results["category"].astype("category")
results["alignment"] = results["alignment"].astype("category")

In [214]:
results.category.unique()

['ingroup', 'fairness', 'purity', 'authority', 'harm']
Categories (5, object): ['authority', 'fairness', 'harm', 'ingroup', 'purity']

#### inverses

In [ ]:
# results['response_adjusted'] = results['response']

In [215]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature,response_processed
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,1.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0


In [216]:
inv = results.loc[results['origin'] == "inverse"]
inv = inv.replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [5.0, 4.0, 3.0, 2.0, 1.0, 0.0])

In [217]:
obv = results.loc[results['origin'] != "inverse"]

In [218]:
z = pd.concat([inv, obv], ignore_index=False)
z= z.sort_index()

In [219]:
z.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature,response_processed
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,1.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0


In [221]:
adjusted = z['response_processed']

In [222]:
results['response_adjusted'] = adjusted

In [223]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,model,temperature,response_processed,response_adjusted
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,1.0,1.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0,5.0


In [224]:
display(results)

,response,prompt,alignment,prompt_number,category,origin,model,temperature,response_processed,response_adjusted
0,1,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,1.0,1.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
3,2,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...
1195,1,"When the government makes laws, it is not the ...",conservative,4.4,fairness,inverse,gpt-4o-mini,2.0,1.0,4.0
1196,"5 берем นักลงทุนสัมพันธ์jsonัพ>(_sb}],imonials...",Men and women can play the same roles in society.,conservative,5.1,authority,inverse,gpt-4o-mini,2.0,5.0,0.0
1197,4,If I were a soldier and disagreed with my comm...,conservative,5.2,authority,inverse,gpt-4o-mini,2.0,4.0,1.0
1198,4,Respect for authority is not something all chi...,conservative,5.3,authority,inverse,gpt-4o-mini,2.0,4.0,1.0


In [225]:
results = results[['response', 'response_processed', 'response_adjusted', 'prompt', 'alignment', 'prompt_number', 'category', 'origin', 'model', 'temperature']]

In [226]:
display(results)

,response,response_processed,response_adjusted,prompt,alignment,prompt_number,category,origin,model,temperature
0,1,1.0,1.0,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,2\ResponseProv_des basesเตย conflict_xcompiler...,2.0,2.0,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,2,2.0,2.0,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,2.0,2.0,The government should strive to improve the we...,liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,5.0,5.0,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,gpt-4o-mini,2.0
...,...,...,...,...,...,...,...,...,...,...
1195,1,1.0,4.0,"When the government makes laws, it is not the ...",conservative,4.4,fairness,inverse,gpt-4o-mini,2.0
1196,"5 берем นักลงทุนสัมพันธ์jsonัพ>(_sb}],imonials...",5.0,0.0,Men and women can play the same roles in society.,conservative,5.1,authority,inverse,gpt-4o-mini,2.0
1197,4,4.0,1.0,If I were a soldier and disagreed with my comm...,conservative,5.2,authority,inverse,gpt-4o-mini,2.0
1198,4,4.0,1.0,Respect for authority is not something all chi...,conservative,5.3,authority,inverse,gpt-4o-mini,2.0


In [227]:
results.to_csv(f'gpt4omini_temp2_base_results_PROCESSED.csv', index = True)